# 用Paddlepaddle做一个凡尔赛文学生成器

## 本文禁止转载~

之前微博上掀起了一股装逼文体的新热潮

该文体先抑后扬

​将装逼隐藏在浮于表面的抱怨之中

装逼者总是在不经意间流露出贵族式的烦恼

看似抱怨

实则炫耀

这样的写作手法被称作——

凡尔赛文学

![](https://ai-studio-static-online.cdn.bcebos.com/bfd834bb278946d8a1c6370809612a2d350d2a5ec9af4beca7c2a250310981e9)

![](https://ai-studio-static-online.cdn.bcebos.com/e74f871faf3e4e0ca444c9e2bd73b7a539f78c7002d048bb92ba31f93c8617f7)


本项目运用Paddlehub实现了根据关键词的凡尔赛文学自动生成器。

![](https://ai-studio-static-online.cdn.bcebos.com/8206ec3b0f654ec2b18a2867a7b6be7b7f3e9d93d60b4b78bb981eec0d1570c8)


## 1. 安装相关库

In [ ]:
! pip install xlrd
! pip install --upgrade paddlehub
! pip install paddle-ernie

按照数据集要求对其进行整理，格式为“序号\t输入文本\t标签”

In [ ]:
import pandas as pd

df = pd.read_excel("data.xlsx")
Keys = df["关键词"].values
Txts = df["文案"].values

with open("format_data.txt", "w") as f:
    for i, k in enumerate(Keys):
        t = Txts[i]
        # t = "凡尔赛"
        f.write("{}\t{}\t{}\n".format(i, k, t))

## 2. 调用Paddlehub模型进行预训练

### 模型我们选用ERNIE-GEN模型

论文地址：https://arxiv.org/abs/2001.11314

![](https://ai-studio-static-online.cdn.bcebos.com/9698f8fef8004f579d01ca0d4a8a8255d58a8cc895c44d59b97141c82a23b833)


In [2]:
import paddlehub as hub

module = hub.Module(name="ernie_gen")

result = module.finetune(
    train_path='format_data.txt',
    save_dir="Versailles_param",
    max_steps=1200,
    noise_prob=0.1,
    save_interval=400,
    max_encode_len=60,
    max_decode_len=60
)

# 将训练参数打包为hub model
module.export(params_path=result['last_save_path'], module_name="Versailles", author="lyp")
!hub install Versailles

[2020-12-10 22:26:30,355] [    INFO] - Installing ernie_gen module


[==================================================] 100.00%
Uncompress /home/aistudio/.paddlehub/tmp/tmp4ioerv1o/ernie_gen
[==================================================] 100.00%


[2020-12-10 22:26:34,769] [    INFO] - Successfully installed ernie_gen-1.0.2
2020-12-10 22:26:34,813-INFO: get pretrain dir from https://ernie-github.cdn.bcebos.com/model-ernie1.0.1.tar.gz
downloading https://ernie-github.cdn.bcebos.com/model-ernie1.0.1.tar.gz: 788478KB [00:14, 53950.68KB/s]                            
[INFO] 2020-12-10 22:26:59,170 [feature_column.py:  349]:	reading raw files from format_data.txt
2020-12-10 22:26:59,174-INFO: get pretrain dir from https://ernie-github.cdn.bcebos.com/model-ernie1.0.1.tar.gz
2020-12-10 22:27:02,458-INFO: loading pretrained model from /tmp/466eabcffd6d6a83ae9cb97dd1a167bd
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:718: UserWarning: Varibale [ pooler.weight pooler.bias ] are not used, because not included in layers state_dict
  format(" ".join(unused_para_list)))
[2020-12-10 22:27:09,748] [    INFO] - [step 0 / 1200]train loss 6.70396, ppl 815.63086, elr 0.000e+00
[2020-12-10 22:27:3

2020-12-10 22:31:38,676-INFO: font search path ['/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf', '/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/afm', '/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/pdfcorefonts']
2020-12-10 22:31:39,012-INFO: generated new fontManager


## 3. 运行预测

In [ ]:
import paddlehub as hub

module = hub.Module(directory="/home/aistudio/Versailles/")

In [4]:
test_texts = ["凡尔赛"]
results = module.generate(texts=test_texts, use_gpu=False, beam_width=10)
for result in results[0]:
    print(result)

今天又是努力搬砖的一天！根本没时间摸鱼，中午只能休息两个小时，虽然五点就能准时下班，但早上十点就得到
今天又是努力搬砖的一天！根本没时间摸鱼中午只能休息两个小时虽然五点就能准时下班但早上十点就得到公司还
今天看中了一栋别墅，我真的好喜欢位置也好，跑到楼顶就能看见天安门。但是真的太贵啦，买下它要花掉我一个
今天看中了一栋别墅,我真的好喜欢位置也好,跑到楼顶就能看见天安门。但是真的太贵啦,买下它要花掉我一个
真羡慕那些可以随随便便离家出走的孩子，我都出来一个月了，还没走出我家草坪。
真羡慕年轻人啊，活力满满见个冠军奖杯那么激动，不像我，拿好几次都没啥感觉了，上次还差点手滑给摔了。
今天看中了一栋别墅，我真的好喜欢位置也好，跑到楼顶就能看见天安门，但是真的太贵啦，买下它要花掉我一个
我在沙发上背单词，突然牛奶热好了我去拿，回来的时候发现没加书签。我问他：“我背到哪了？”他不慌不忙地
今天又是努力搬砖的一天！根本没时间摸鱼中午只能休息两个小时虽然五点就能准时下班但早上十点就得到公司，
真羡慕那些可以随随便便离家出走的孩子，我都出来一个月了，还没走出我家草坪


In [5]:
test_texts = ["房"]
results = module.generate(texts=test_texts, use_gpu=False, beam_width=5)
for result in results[0]:
    print(result)

最近去试完衣服顺路回家都买几捧玫瑰，老公突然就说要买一个带院子的房子，种满玫瑰叫园丁专门打理，但是玫
难受了，我竟然错过了悉尼歌剧院的演出！因为他非得拉着我去挑什么房子，一个千佛山脚下的普通别墅而已，至
最近去试完衣服顺路回家都买几捧玫瑰，老公突然就说要买一个带院子的房子，种满玫瑰叫园丁专门打理。
“最近有个追求者给我在陆家嘴买了套房但是我不想要我已经在汤臣一品有三层了我觉得他是在用钱侮辱我呜呜呜
难受了，我竟然错过了悉尼歌剧院的演出！因为他非得拉着我去挑什么房子。


In [6]:
test_texts = ["老公"]
results = module.generate(texts=test_texts, use_gpu=False, beam_width=5)
for result in results[0]:
    print(result)

今天皮带忘带了去gucci随便买了个结果打孔的时候店员说腰太细了她从来没打过这么细的我觉得女生圆润一点好的呀
“今天皮带忘带了去gucci随便买了个结果打孔的时候店员说腰太细了她从来没打过这么细的我觉得女生圆润一点好的
今天皮带忘带了去gucci随便买了个结果打孔的时候店员说腰太细了她从来没打过这么近的我觉得女生圆润一点好的呀
“今天皮带忘带了去gucci随便买了个结果打孔的时候店员说腰太细了她从来没打过这么近的我觉得女生圆润一点好的
最近老公发了个朋友圈，说特朗普落选那天全朋友圈消费由他买单，虽然我知道他是开玩笑的，但是他那帮朋友真


## 4. 改进方向：

可以看到数据集太少，出现了明显的过拟合现象，可以多搜集一些。